In [14]:
pip install juliacall

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install quri-parts

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install quri-parts-itensor

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install quri-parts-qiskit

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install quri-parts-openfermion

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install qiskit-nature

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install qiskit==0.41.1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 50.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.3/241.3 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 5.8 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 69.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 21.4 MB/s eta 0:00:00
  Created wheel for qiskit: filename=qiskit-0.41.1-py3-none-any.whl size=12896 sha256=65710a41ff8c3fea7c5158729b8407ffbe0fe20515344e

In [7]:
pip install openfermion

Note: you may need to restart the kernel to use updated packages.


In [1]:
import qiskit
from qiskit.quantum_info import SparsePauliOp, Operator

import numpy as np

from qiskit.circuit import Parameter, Gate
#from qiskit import QuantumCircuit
from scipy.optimize import minimize
from qiskit.circuit import QuantumCircuit, ParameterVector
from scipy.linalg import expm
from tqdm import tqdm
#from qiskit.circuit.library import HamiltonianGate
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper
import marshal
import csv

In [2]:
import sys

sys.path.append("../")

from openfermion.transforms import jordan_wigner

from quri_parts.algo.ansatz import SymmetryPreservingReal
from quri_parts.algo.optimizer import SPSA, OptimizerStatus
from quri_parts.circuit import LinearMappedUnboundParametricQuantumCircuit
from quri_parts.core.estimator.gradient import parameter_shift_gradient_estimates
from quri_parts.core.measurement import bitwise_commuting_pauli_measurement, CachedMeasurementFactory
from quri_parts.core.sampling.shots_allocator import (
    create_equipartition_shots_allocator,
)
from quri_parts.core.state import ParametricCircuitQuantumState, ComputationalBasisState, GeneralCircuitQuantumState

from quri_parts.openfermion.operator import operator_from_openfermion_op



from utils.challenge_2024 import ChallengeSampling, ExceededError, problem_hamiltonian

# define challenge_sampling
challenge_sampling = ChallengeSampling()

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


   Resolving package versions...
  No Changes to `~/anaconda3/envs/JupyterSystemEnv/julia_env/Project.toml`
  No Changes to `~/anaconda3/envs/JupyterSystemEnv/julia_env/Manifest.toml`


test mps sampling took: (56.44288611412048, Counter({2: 6, 0: 4}))


In [3]:
from quri_parts.qiskit.circuit import circuit_from_qiskit

# create estimator
shots_allocator = create_equipartition_shots_allocator()
measurement_factory = bitwise_commuting_pauli_measurement

estimator = challenge_sampling.create_concurrent_parametric_sampling_estimator(
    n_shots=1000,
    measurement_factory=measurement_factory,
    shots_allocator=shots_allocator,
)

# Su(n) gate and ansatz

In [4]:
from quri_parts.circuit import UnboundParametricQuantumCircuit

def su4(qubits,circuit):
    

    circuit.add_ParametricRY_gate(qubits[0])
    circuit.add_ParametricRX_gate(qubits[0])
    circuit.add_ParametricRY_gate(qubits[0])
    
    circuit.add_ParametricRY_gate(qubits[1])
    circuit.add_ParametricRX_gate(qubits[1])
    circuit.add_ParametricRY_gate(qubits[1])
    
    circuit.add_CNOT_gate(qubits[0],qubits[1])
    
    circuit.add_ParametricRZ_gate(qubits[0])
    circuit.add_ParametricRY_gate(qubits[1])

    circuit.add_CNOT_gate(qubits[1],qubits[0])
    
    circuit.add_ParametricRY_gate(qubits[1])
    
    circuit.add_CNOT_gate(qubits[0],qubits[1])
    
    circuit.add_ParametricRY_gate(qubits[0])
    circuit.add_ParametricRX_gate(qubits[0])
    circuit.add_ParametricRY_gate(qubits[0])
    
    circuit.add_ParametricRY_gate(qubits[1])
    circuit.add_ParametricRX_gate(qubits[1])
    circuit.add_ParametricRY_gate(qubits[1])

'''
def su4(params,qubits,circuit):
    

    circuit.add_RY_gate(qubits[0], params[0])
    circuit.add_RX_gate(qubits[0], params[1])
    circuit.add_RY_gate(qubits[0], params[2])
    
    circuit.add_RY_gate(qubits[1], params[3])
    circuit.add_RX_gate(qubits[1], params[4])
    circuit.add_RY_gate(qubits[1], params[5])
    
    circuit.add_CNOT_gate(qubits[0],qubits[1])
    
    circuit.add_RZ_gate(qubits[0], params[6])
    circuit.add_RY_gate(qubits[1], params[7])

    circuit.add_CNOT_gate(qubits[1],qubits[0])
    
    circuit.add_RY_gate(qubits[1], params[8])
    
    circuit.add_CNOT_gate(qubits[0],qubits[1])
    
    circuit.add_RY_gate(qubits[0], params[9])
    circuit.add_RX_gate(qubits[0], params[10])
    circuit.add_RY_gate(qubits[0], params[11])
    
    circuit.add_RY_gate(qubits[1], params[12])
    circuit.add_RX_gate(qubits[1], params[13])
    circuit.add_RY_gate(qubits[1], params[14])
    
'''    
''' 
def su4(params):
    q = qiskit.QuantumRegister(2)
    cir = qiskit.QuantumCircuit(q)
    
    cir.ry(params[0], 0)
    cir.rx(params[1], 0)
    cir.ry(params[2], 0)
    # Single U(2) parameterization on qubit 2
    cir.ry(params[3], 1)
    cir.rx(params[4], 1)
    cir.ry(params[5], 1)
    # CNOT with control on qubit 2
    cir.cx(1, 0)
    # Rz and Ry gate
    cir.rz(params[6], 0)
    cir.ry(params[7], 1)
    # CNOT with control on qubit 1
    cir.cx(0, 1)
    # Ry gate on qubit 2
    cir.ry(params[8], 1)
    # CNOT with control on qubit 2
    cir.cx(1, 0)
    # Single U(2) parameterization on qubit 1
    cir.ry(params[9], 0)
    cir.rx(params[10], 0)
    cir.ry(params[11], 0)
    # Single U(2) parameterization on qubit 2
    cir.ry(params[12], 1)
    cir.rx(params[13], 1)
    cir.ry(params[14], 1)
    
    return cir
'''

' \ndef su4(params):\n    q = qiskit.QuantumRegister(2)\n    cir = qiskit.QuantumCircuit(q)\n    \n    cir.ry(params[0], 0)\n    cir.rx(params[1], 0)\n    cir.ry(params[2], 0)\n    # Single U(2) parameterization on qubit 2\n    cir.ry(params[3], 1)\n    cir.rx(params[4], 1)\n    cir.ry(params[5], 1)\n    # CNOT with control on qubit 2\n    cir.cx(1, 0)\n    # Rz and Ry gate\n    cir.rz(params[6], 0)\n    cir.ry(params[7], 1)\n    # CNOT with control on qubit 1\n    cir.cx(0, 1)\n    # Ry gate on qubit 2\n    cir.ry(params[8], 1)\n    # CNOT with control on qubit 2\n    cir.cx(1, 0)\n    # Single U(2) parameterization on qubit 1\n    cir.ry(params[9], 0)\n    cir.rx(params[10], 0)\n    cir.ry(params[11], 0)\n    # Single U(2) parameterization on qubit 2\n    cir.ry(params[12], 1)\n    cir.rx(params[13], 1)\n    cir.ry(params[14], 1)\n    \n    return cir\n'

In [5]:
def apply_su4_in_layers(depth, nqubits, fix_pars_per_layer=False):
    """Apply a (callable) operation in layers.

    Args:
        theta (tensor_like): The arguments passed to the operations. The expected shape
            is ``(depth, k, num_params_op)``, where ``k`` is determined by ``fix_pars_per_layer``
            and ``num_params_op`` is the number of paramters each operation takes.
        Op (callable): The operation to apply
        depth (int): The number of layers to apply
        fix_pars_per_layer (bool): Whether or not all operations applied in parallel share the
            same parameters. If ``True``, the dimension ``k`` in the shape of ``theta`` is set to
            two, otherwise it is set to ``nqubits``.

    """
    circuit = UnboundParametricQuantumCircuit(nqubits)
    #circuit = LinearMappedUnboundParametricQuantumCircuit(n_qubits)
    qubits = list(range(nqubits))
    
    if fix_pars_per_layer:
        size = 2
    else:
        size = nqubits
    
    for d in range(depth):
        # Even-odd qubit pairs
        idx = 0
        for q in qubits[0::2]:
            #`su4(theta[(d*size + idx)*15:(d*size + idx +1)*15], [q, (q+1+d)%nqubits],circuit)
            su4([q, (q+1+d)%nqubits],circuit)
            if q == nqubits - 2:
                idx += 1
            elif not fix_pars_per_layer:
                idx += 1
            
        # Odd-even qubit pairs
        for q in qubits[1::2]:
            # su4(theta[(d*size + idx)*15:(d*size + idx +1)*15], [q, (q+1+d)%nqubits],circuit)
            su4([q, (q+1+d)%nqubits],circuit)
            if not fix_pars_per_layer:
                idx += 1
    return circuit

# VQE

In [ ]:
def print_cpu_usage():
  import psutil
  split_bar = '='*20
  memory_info = psutil.virtual_memory()._asdict()
  print(f"{split_bar} Memory Usage {split_bar}")
  for k,v in memory_info.items():
    print(k, v)
  print(f"{split_bar} CPU Usage {split_bar}")
  print(f"CPU percent: {psutil.cpu_percent()}%")

In [11]:
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_algorithms.gradients import FiniteDiffEstimatorGradient
import math

def vqe(
    num_qubits,
    observable,
    depth,
    optimizer,
    max_steps,
    init_param=None):

    # n = 4**num_SUgate_qubits - 1 = 15
    param_len = depth*num_qubits*15


    # List to store the minimized values after every iteration
    '''
    num_orbitals = num_qubits // 2
    num_particles = num_qubits // 2
    num_alpha = num_particles // 2
    num_beta = num_particles // 2

    mapper = JordanWignerMapper()
    init_state = HartreeFock(num_orbitals, (num_alpha, num_beta) , qubit_mapper=mapper)

    circ = init_state.compose(apply_su4_in_layers(params, depth, num_qubits, fix_pars_per_layer=False))
    quri_parts_circuit = circuit_from_qiskit(circ)
    circuit_state = GeneralCircuitQuantumState(num_qubits, quri_parts_circuit)
    '''
    n_site = num_qubits // 2
    hf_gates = ComputationalBasisState(num_qubits, bits=2**n_site - 1).circuit.gates
    circuit = UnboundParametricQuantumCircuit(num_qubits).combine(
            hf_gates
        )
    circuit.extend(apply_su4_in_layers(depth, num_qubits, fix_pars_per_layer=False))
    circuit_state = ParametricCircuitQuantumState(num_qubits, circuit)
    
    def cost_function(params):
        """Cost function of the VQE."""

        # Initialize Hartree-Fock state
        
        estimated_value = estimator(hamiltonian,circuit_state,[params])[0].value.real

        # Store the minimized value
        # minimized_values.append(estimator_expvals)
        # print("Minimized Exp Value: ", estimator_expvals)
        # print("-----------------------------------------")

        return estimated_value
    
    def grad_function(params):
        grad = parameter_shift_gradient_estimates(
            observable, circuit_state, params, estimator
        )
        return np.asarray([i.real for i in grad.values])
    '''
    def grad_function(params):
        """ use FiniteDiffEstimatorGradient """

        # Initialize Hartree-Fock state
        num_orbitals = num_qubits // 2
        num_particles = num_qubits // 2
        num_alpha = num_particles // 2
        num_beta = num_particles // 2

        mapper = JordanWignerMapper()
        init_state = HartreeFock(num_orbitals, (num_alpha, num_beta) , qubit_mapper=mapper)

       
        # This could be optimized
        indices = [*range(len(params))]

        offset = np.identity(len(params))[indices, :]
        plus = params + 0.001 * offset
        minus = params - 0.001 * offset

        # length of circ_list is param_len
        quri_circ_plus = [circuit_from_qiskit(init_state.compose(apply_su4_in_layers(params_value, depth, num_qubits, fix_pars_per_layer=False))) for params_value in plus]
        quri_circ_minus = [circuit_from_qiskit(init_state.compose(apply_su4_in_layers(params_value, depth, num_qubits, fix_pars_per_layer=False))) for params_value in minus]
        
        plus_state = [GeneralCircuitQuantumState(num_qubits, circuit) for circuit in quri_circ_plus]
        minus_state = [GeneralCircuitQuantumState(num_qubits, circuit) for circuit in quri_circ_minus]

        
        estimated_value_plus = [plusresult.value.real for plusresult in estimator([observable] * param_len, plus_state)]
        estimated_value_minus = [minusresult.value.real for minusresult in estimator([observable] * param_len, minus_state)]


        return (estimated_value_plus-estimated_value_minus)/0.002
        #return circ_plus_list
    '''

    # Create initial parameters
    if init_param is None:
        init_params = np.array([0.1]*param_len)
    else:
        init_params = init_param

    Optimizer = optimizer(maxiter=max_steps)
    # Optimizer = optimizer(maxfun=max_steps)
    opt_result = Optimizer.minimize(fun=cost_function, x0=init_params, jac=grad_function)

    return opt_result

In [ ]:
def separat_vqe(
    num_qubits,
    observable,
    depth,
    optimizer,
    max_steps,
    separat_steps,
    init_param=None):
    
    with open("vqe_params_" + datetime + ".csv", "a", newline='') as fd:
        statewriter = csv.writer(fd, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        if init_param is None:
            result = vqe(num_qubits,
                         observable,
                         depth,
                         optimizer,
                         max_steps=separat_steps,
                         init_param=None)
        else:
            result = vqe(num_qubits,
                         observable,
                         depth,
                         optimizer,
                         max_steps=separat_steps,
                         init_param=init_param)
        print(result.fun)
        statewriter.writerow(result.x)
    
    
        for i in range(separat_steps,max_steps,separat_steps):
            result = vqe(num_qubits,
                         observable,
                         depth,
                         optimizer,
                         separat_steps,
                         init_param=result.x)
            print(result.fun)
            statewriter.writerow(result.x)
            print("completed steps:",i)
        

# problem hamiltonian

In [12]:
n_qubits = 4
seed=0
hamiltonian_directory="../hamiltonian"
ham = problem_hamiltonian(n_qubits, seed, hamiltonian_directory)
n_site = n_qubits // 2
total_shots = 10**6
jw_hamiltonian = jordan_wigner(ham)

hamiltonian = operator_from_openfermion_op(jw_hamiltonian)

04qubits_00


# Train

In [13]:
from qiskit_algorithms.optimizers import SLSQP, P_BFGS, NELDER_MEAD
import time
start = time.time()
test = vqe(n_qubits,
           observable=hamiltonian,
           depth=1,
           optimizer=SLSQP,
           max_steps=10,
           init_param=None)
'''
testt = separat_vqe(
    n_qubits,
    observable=hamiltonian,
    depth=2,
    optimizer=SLSQP,
    max_steps=100,
    separat_steps=20,
    init_param=None)
'''
end = time.time()
print("Time Taken: ", end-start)

Time Taken:  198.1214702129364


In [14]:
test.fun

-3.944999999999999